In [ ]:
%%capture
!pip install -U sentence-transformers rank_bm25


In [ ]:
import gzip
import os
import torch
import time
import warnings
import pickle
#from google.colab import drive
import numpy as np
import pandas as pd
from itertools import chain
from sentence_transformers import SentenceTransformer, CrossEncoder, util
warnings.filterwarnings('ignore')
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dfs = []
for file in os.listdir('/Corpus/json_paragrafo/'):
    if file.endswith("json"): 
        file = '/Corpus/json_paragrafo/' + file
        print(file)
        data = pd.read_json(file)
        dfs.append(data)
df = pd.concat(dfs, ignore_index=True)

/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_30_fev_mar_de_2013.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_75_out_nov_de_2020.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_16_out_nov_2010.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_61_mai_jun_de_2018.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_05_nov_dez_de_2008.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_66_mar_abr_de_2019.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_07_mar_abr_de_2009.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_02_maio_junho_de_2008.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_45_set_out_de_2015.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_67_mai_jun_de_2019.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_12_fev_mar_de_2010.json
/content/drive/MyDrive/IC_Lucas/Sorria/json_paragrafo/Sorria_34_o

In [ ]:
df.head()

,section,title,author,edition,months,year,text
0,CARTA DA ROBERTA,conversas sinceras,,30,fev/mar,2013,E se tudo O QUE FALTA PARA SUA VIDA ﬁcar compl...
1,PRAZERES SIMPLES,alegrias do dia a dia,DILSON BRANCO,30,fev/mar,2013,"É hora! É HORA! A DATA OFICIAL, todo mundo sab..."
2,COMO EU FAÇO?,manual de boas ideias,SOLANGE MEDEIROS,30,fev/mar,2013,"Dá uma carona? Para ir para o trabalho, para a..."
3,DÁ PRA MUDAR?,"um repórter, um desaﬁo",ANITA MARTINS,30,fev/mar,2013,Des conectada NOSSA REPÓRTER PASSOU 15 DIAS SE...
4,LUGAR EM COMUM,conviver e compartilhar,RAFAELA CARVALHO,30,fev/mar,2013,Para ir e vir POR QUE SOMOS TÃO VIOLENTOS NO T...


In [ ]:
texto = "".join(df['text'].tolist())
corpus = texto.split('$$')


In [ ]:
len(corpus)

21262

In [ ]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(corpus):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)


  0%|          | 0/21262 [00:00<?, ?it/s]

In [ ]:
# This function will search all wikipedia articles for corpus that
# answer the query
def search(query):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    print("Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], corpus[hit['corpus_id']].replace("\n", " ")))

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant corpus
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved corpus with the cross_encoder
    cross_inp = [[query, corpus[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], corpus[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:10]:  
        print("\t{:.3f}\t{}".format(hit['cross-score'], corpus[hit['corpus_id']].replace("\n", " ")))


# bert-base-cased-squad-v1.1-portuguese + pierreguillou/bert-base-cased-squad-v1.1-portuguese

In [ ]:
if not torch.cuda.is_available():
    raise SystemError('No GPU found. Please add GPU to your notebook')

#We use the Bi-Encoder to encode all corpus, so that we can use it with sematic search
bi_encoder = SentenceTransformer('pierreguillou/bert-base-cased-squad-v1.1-portuguese')
top_k = 50                          #Number of corpus we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('neuralmind/bert-large-portuguese-cased')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)



Downloading:   0%|          | 0.00/862 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/862 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/pierreguillou_bert-base-cased-squad-v1.1-portuguese. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/pierreguillou_bert-base-cased-squad-v1.1-portuguese were not used when initializing BertModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/pierreguillou_bert-base-cased-squa

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from th

Batches:   0%|          | 0/665 [00:00<?, ?it/s]

In [ ]:
search(query = "O que é amizade?")

Input question: O que é amizade?
Top-3 lexical search (BM25) hits
	13.619	 Assim que conseguir, faça um esforço para que a amizade se mantenha: promova reuniões, partipe das conversas, mande felicitações nos aniversários e sempre cumpra o que prometeu
	12.666	 O esporte fortaleceu tanto a amizade do grupo que os amigos até já aproveitaram um Carnaval para jogar bola fantasiados de mulher. “Quando você pratica um esporte que ama com os companheiros, tudo é alegria”, conta Rafael
	12.437	 Parece que está falando sobre mim. Fala sobre ver o mundo devagar, com olhos curiosos, conversar com pessoas que não conhece e fazer amizade com a dona da lojinha na esquina de um lugar que você nunca foi...
	12.217	 os pesquisadores pediam que estimassem quão íngreme era a subida e lhes entregavam uma mochila pesada. Os participantes acompanhados apostaram em valores mais baixos do que os que estavam sozinhos – ao lado de um amigo, o obstáculo era menos intimidante. E, quanto mais longa a amizade, meno

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.973	 Como a preparou?”
	0.961	 VAI ENCARAR? O mundo virtual é semelhante ao real
	0.960	 4. Você ronca? a) Nunca ronquei na vida
	0.960	 vida? Para funcionar, seja sincero
	0.960	 Mas quer saber? Não gostou tanto assim
	0.959	 5o Conversar não resolveu? O primeiro encontrá-lo facilmente na internet
	0.958	 Onde começar? Quase todas as academias oferecem aulas de step
	0.958	 trabalhar para viver melhor Amigoé pra Aumento? Desafios? Não
	0.957	 É BOM PRA QUÊ? Estimula a capacidade de concentração e desenvolve a coordenação motora e o equilíbrio
	0.957	 Parceiros de luta? Amor à causa? a inércia com uma ação realista

-------------------------

Top-3 Cross-Encoder Re-ranker hits
	0.614	 trabalhar para viver melhor Amigoé pra Aumento? Desafios? Não
	0.592	 Parceiros de luta? Amor à causa? a inércia com uma ação realista
	0.536	 Relaciona a liberdade à interdependência: Re “Podemos realmente viver um sem o outro?”
	0.529	 vida?

In [ ]:
search(query = "O que é amor?")

Input question: O que é amor?
Top-3 lexical search (BM25) hits
	13.645	 É o amor que nos inspira a dar o melhor de nós e a esperar o melhor para os outros
	13.390	 Com ele, criamos vínculos capazes de superar diferenças e distâncias, de vencer o peso do tempo e do cansaço. O amor que recebemos — da família, dos amigos, das paixões, de nós mesmos — nos torna mais fortes, equilibrados, generosos e felizes. O amor que doamos também. Por amor, arriscamos no incerto e acreditamos no futuro
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.130	 Mas se o amor é universal, a forma como o transformamos em relacionamentos é particular: cada um tem a sua, fruto de sua história, do seu tempo, da pessoa ao lado. Enquanto na família ou entre as amizades encaramos o amor de forma tranqüila e espontânea, tudo muda quando se trata do sentiment

In [ ]:
search(query = "como fazer biscoitos?")

Input question: como fazer biscoitos?
Top-3 lexical search (BM25) hits
	12.413	 Se você for fazer vários biscoitos, uma dica do diâmetro do círculo) e cole-o no centro
	9.724	 não existe nada mais fácil que fazer biscoitos, e por isso mesmo as crianças adoram ajudar. “elas podem usar copinhos para modelar a massa”, sugere a doceira Giuliana Azambuja. “ou fazer bolinhas e usar um garfo para decorá-las.” os pequenos também podem participar acrescentando ingredientes à mistura, como pedacinhos de chocolate. “A ajuda na cozinha vira uma grande diversão”, diz Giuliana
	9.466	 Algumas sugestões: biscoitos integrais, iogurte com granola ou um sanduíche
	9.337	 Quebre os biscoitos em pedaços menores e umedeça-os com a bebida escolhida
	9.295	 Você precisa de: > Batatas cruas > 1 faca sem ponta > Cortadores de biscoitos com formatos variados (desses que parecem forminhas) > Tinta de tecido, guache ou almofada para carimbos de várias cores > Papel ou tecido para estampar Como fazer: 1. Corte a b

#  paraphrase-multilingual-mpnet-base-v2  + cross-encoder/ms-marco-MiniLM-L-6-v2

In [ ]:
if not torch.cuda.is_available():
    raise SystemError('No GPU found. Please add GPU to your notebook')

#We use the Bi-Encoder to encode all corpus, so that we can use it with sematic search
bi_encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
top_k = 50                          #Number of corpus we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)



Batches:   0%|          | 0/665 [00:00<?, ?it/s]

In [ ]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(corpus):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)


  0%|          | 0/21262 [00:00<?, ?it/s]

In [ ]:
search(query = "O que é amizade?")

Input question: O que é amizade?
Top-3 lexical search (BM25) hits
	13.619	 Assim que conseguir, faça um esforço para que a amizade se mantenha: promova reuniões, partipe das conversas, mande felicitações nos aniversários e sempre cumpra o que prometeu
	12.666	 O esporte fortaleceu tanto a amizade do grupo que os amigos até já aproveitaram um Carnaval para jogar bola fantasiados de mulher. “Quando você pratica um esporte que ama com os companheiros, tudo é alegria”, conta Rafael
	12.437	 Parece que está falando sobre mim. Fala sobre ver o mundo devagar, com olhos curiosos, conversar com pessoas que não conhece e fazer amizade com a dona da lojinha na esquina de um lugar que você nunca foi...
	12.217	 os pesquisadores pediam que estimassem quão íngreme era a subida e lhes entregavam uma mochila pesada. Os participantes acompanhados apostaram em valores mais baixos do que os que estavam sozinhos – ao lado de um amigo, o obstáculo era menos intimidante. E, quanto mais longa a amizade, meno

In [ ]:
search(query = "O que é amor?")

Input question: O que é amor?
Top-3 lexical search (BM25) hits
	13.645	 É o amor que nos inspira a dar o melhor de nós e a esperar o melhor para os outros
	13.390	 Com ele, criamos vínculos capazes de superar diferenças e distâncias, de vencer o peso do tempo e do cansaço. O amor que recebemos — da família, dos amigos, das paixões, de nós mesmos — nos torna mais fortes, equilibrados, generosos e felizes. O amor que doamos também. Por amor, arriscamos no incerto e acreditamos no futuro
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.130	 Mas se o amor é universal, a forma como o transformamos em relacionamentos é particular: cada um tem a sua, fruto de sua história, do seu tempo, da pessoa ao lado. Enquanto na família ou entre as amizades encaramos o amor de forma tranqüila e espontânea, tudo muda quando se trata do sentiment

In [ ]:
search(query = "como fazer biscoitos?")

Input question: como fazer biscoitos?
Top-3 lexical search (BM25) hits
	12.413	 Se você for fazer vários biscoitos, uma dica do diâmetro do círculo) e cole-o no centro
	9.724	 não existe nada mais fácil que fazer biscoitos, e por isso mesmo as crianças adoram ajudar. “elas podem usar copinhos para modelar a massa”, sugere a doceira Giuliana Azambuja. “ou fazer bolinhas e usar um garfo para decorá-las.” os pequenos também podem participar acrescentando ingredientes à mistura, como pedacinhos de chocolate. “A ajuda na cozinha vira uma grande diversão”, diz Giuliana
	9.466	 Algumas sugestões: biscoitos integrais, iogurte com granola ou um sanduíche
	9.337	 Quebre os biscoitos em pedaços menores e umedeça-os com a bebida escolhida
	9.295	 Você precisa de: > Batatas cruas > 1 faca sem ponta > Cortadores de biscoitos com formatos variados (desses que parecem forminhas) > Tinta de tecido, guache ou almofada para carimbos de várias cores > Papel ou tecido para estampar Como fazer: 1. Corte a b

# bertimbau + pierreguillou/bert-base-cased-squad-v1.1-portuguese

In [ ]:
if not torch.cuda.is_available():
    raise SystemError('No GPU found. Please add GPU to your notebook')

#We use the Bi-Encoder to encode all corpus, so that we can use it with sematic search
bi_encoder = SentenceTransformer('neuralmind/bert-large-portuguese-cased')
top_k = 50                          #Number of corpus we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('pierreguillou/bert-base-cased-squad-v1.1-portuguese')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)



No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/neuralmind_bert-large-portuguese-cased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/neuralmind_bert-large-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForS

Some weights of the model checkpoint at pierreguillou/bert-base-cased-squad-v1.1-portuguese were not used when initializing BertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pierreguillou/bert-base-cased-squad-v1.1-portuguese and are newly initialized: ['bert.pooler.dense.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a

Batches:   0%|          | 0/665 [00:00<?, ?it/s]

In [ ]:
search(query = "O que é amizade?")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Input question: O que é amizade?
Top-3 lexical search (BM25) hits
	13.619	 Assim que conseguir, faça um esforço para que a amizade se mantenha: promova reuniões, partipe das conversas, mande felicitações nos aniversários e sempre cumpra o que prometeu
	12.666	 O esporte fortaleceu tanto a amizade do grupo que os amigos até já aproveitaram um Carnaval para jogar bola fantasiados de mulher. “Quando você pratica um esporte que ama com os companheiros, tudo é alegria”, conta Rafael
	12.437	 Parece que está falando sobre mim. Fala sobre ver o mundo devagar, com olhos curiosos, conversar com pessoas que não conhece e fazer amizade com a dona da lojinha na esquina de um lugar que você nunca foi...
	12.217	 os pesquisadores pediam que estimassem quão íngreme era a subida e lhes entregavam uma mochila pesada. Os participantes acompanhados apostaram em valores mais baixos do que os que estavam sozinhos – ao lado de um amigo, o obstáculo era menos intimidante. E, quanto mais longa a amizade, meno

In [ ]:
search(query = "O que é amor?")

Input question: O que é amor?
Top-3 lexical search (BM25) hits
	13.645	 É o amor que nos inspira a dar o melhor de nós e a esperar o melhor para os outros
	13.390	 Com ele, criamos vínculos capazes de superar diferenças e distâncias, de vencer o peso do tempo e do cansaço. O amor que recebemos — da família, dos amigos, das paixões, de nós mesmos — nos torna mais fortes, equilibrados, generosos e felizes. O amor que doamos também. Por amor, arriscamos no incerto e acreditamos no futuro
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.130	 Mas se o amor é universal, a forma como o transformamos em relacionamentos é particular: cada um tem a sua, fruto de sua história, do seu tempo, da pessoa ao lado. Enquanto na família ou entre as amizades encaramos o amor de forma tranqüila e espontânea, tudo muda quando se trata do sentiment

In [ ]:
search(query = "como fazer biscoitos?")

Input question: como fazer biscoitos?
Top-3 lexical search (BM25) hits
	12.413	 Se você for fazer vários biscoitos, uma dica do diâmetro do círculo) e cole-o no centro
	9.724	 não existe nada mais fácil que fazer biscoitos, e por isso mesmo as crianças adoram ajudar. “elas podem usar copinhos para modelar a massa”, sugere a doceira Giuliana Azambuja. “ou fazer bolinhas e usar um garfo para decorá-las.” os pequenos também podem participar acrescentando ingredientes à mistura, como pedacinhos de chocolate. “A ajuda na cozinha vira uma grande diversão”, diz Giuliana
	9.466	 Algumas sugestões: biscoitos integrais, iogurte com granola ou um sanduíche
	9.337	 Quebre os biscoitos em pedaços menores e umedeça-os com a bebida escolhida
	9.295	 Você precisa de: > Batatas cruas > 1 faca sem ponta > Cortadores de biscoitos com formatos variados (desses que parecem forminhas) > Tinta de tecido, guache ou almofada para carimbos de várias cores > Papel ou tecido para estampar Como fazer: 1. Corte a b

#  paraphrase-multilingual-mpnet-base-v2  + pierreguillou/bert-base-cased-squad-v1.1-portuguese

In [ ]:
if not torch.cuda.is_available():
    raise SystemError('No GPU found. Please add GPU to your notebook')

#We use the Bi-Encoder to encode all corpus, so that we can use it with sematic search
bi_encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
top_k = 50                          #Number of corpus we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('pierreguillou/bert-base-cased-squad-v1.1-portuguese')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)



Some weights of the model checkpoint at pierreguillou/bert-base-cased-squad-v1.1-portuguese were not used when initializing BertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pierreguillou/bert-base-cased-squad-v1.1-portuguese and are newly initialized: ['bert.pooler.dense.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a

Batches:   0%|          | 0/665 [00:00<?, ?it/s]

In [ ]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.

from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for passage in tqdm(corpus):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)


  0%|          | 0/21262 [00:00<?, ?it/s]

In [ ]:
search(query = "O que é amizade?")

Input question: O que é amizade?
Top-3 lexical search (BM25) hits
	13.619	 Assim que conseguir, faça um esforço para que a amizade se mantenha: promova reuniões, partipe das conversas, mande felicitações nos aniversários e sempre cumpra o que prometeu
	12.666	 O esporte fortaleceu tanto a amizade do grupo que os amigos até já aproveitaram um Carnaval para jogar bola fantasiados de mulher. “Quando você pratica um esporte que ama com os companheiros, tudo é alegria”, conta Rafael
	12.437	 Parece que está falando sobre mim. Fala sobre ver o mundo devagar, com olhos curiosos, conversar com pessoas que não conhece e fazer amizade com a dona da lojinha na esquina de um lugar que você nunca foi...
	12.217	 os pesquisadores pediam que estimassem quão íngreme era a subida e lhes entregavam uma mochila pesada. Os participantes acompanhados apostaram em valores mais baixos do que os que estavam sozinhos – ao lado de um amigo, o obstáculo era menos intimidante. E, quanto mais longa a amizade, meno

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



-------------------------

Top-3 Bi-Encoder Retrieval hits
	0.742	 Para o psicólogo Ailton Amélio, a amizade só é válida se é possível um mostrar ao outro diferentes pontos de vista. “Concordar em tudo não é amizade. É bajulação ou falsidade.” Em outras palavras: amigo é quem nos avisa quando erramos e nos ajuda a acertar. Com tato, cuidado e amor
	0.735	 4. Promova a amizade
	0.725	 Especialista no assunto, a professora do departamento de psicologia da Universidade Federal de Minas Gerais Luciana de Souza tenta racionalizar o conceito. “Amizade é a troca intensa de beijos e abraços, a revelação de um maior número de assuntos com mais profundidade, um relacionamento importante para o desenvolvimento social, emocional e também cognitivo”, diz
	0.686	 Mas não bastam situações-limite para forjar amigos. “O que define um amigo é o compartilhamento de valores, de saberes, de ideias, de experiências”, diz Mauro Koury, coordenador do Grupo de Pesquisa em Antropologia e Sociologia das Emoções

In [ ]:
search(query = "O que é amor?")

Input question: O que é amor?
Top-3 lexical search (BM25) hits
	13.645	 É o amor que nos inspira a dar o melhor de nós e a esperar o melhor para os outros
	13.390	 Com ele, criamos vínculos capazes de superar diferenças e distâncias, de vencer o peso do tempo e do cansaço. O amor que recebemos — da família, dos amigos, das paixões, de nós mesmos — nos torna mais fortes, equilibrados, generosos e felizes. O amor que doamos também. Por amor, arriscamos no incerto e acreditamos no futuro
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.130	 Mas se o amor é universal, a forma como o transformamos em relacionamentos é particular: cada um tem a sua, fruto de sua história, do seu tempo, da pessoa ao lado. Enquanto na família ou entre as amizades encaramos o amor de forma tranqüila e espontânea, tudo muda quando se trata do sentiment

In [ ]:
search(query = "como fazer biscoitos?")

Input question: como fazer biscoitos?
Top-3 lexical search (BM25) hits
	12.413	 Se você for fazer vários biscoitos, uma dica do diâmetro do círculo) e cole-o no centro
	9.724	 não existe nada mais fácil que fazer biscoitos, e por isso mesmo as crianças adoram ajudar. “elas podem usar copinhos para modelar a massa”, sugere a doceira Giuliana Azambuja. “ou fazer bolinhas e usar um garfo para decorá-las.” os pequenos também podem participar acrescentando ingredientes à mistura, como pedacinhos de chocolate. “A ajuda na cozinha vira uma grande diversão”, diz Giuliana
	9.466	 Algumas sugestões: biscoitos integrais, iogurte com granola ou um sanduíche
	9.337	 Quebre os biscoitos em pedaços menores e umedeça-os com a bebida escolhida
	9.295	 Você precisa de: > Batatas cruas > 1 faca sem ponta > Cortadores de biscoitos com formatos variados (desses que parecem forminhas) > Tinta de tecido, guache ou almofada para carimbos de várias cores > Papel ou tecido para estampar Como fazer: 1. Corte a b

# bertimbau + cross-encoder/ms-marco-MiniLM-L-6-v2

In [ ]:
if not torch.cuda.is_available():
    raise SystemError('No GPU found. Please add GPU to your notebook')

#We use the Bi-Encoder to encode all corpus, so that we can use it with sematic search
bi_encoder = SentenceTransformer('neuralmind/bert-large-portuguese-cased')
top_k = 50                          #Number of corpus we want to retrieve with the bi-encoder

#The bi-encoder will retrieve 100 documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)



Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/neuralmind_bert-large-portuguese-cased. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/neuralmind_bert-large-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForS

Batches:   0%|          | 0/665 [00:00<?, ?it/s]

In [ ]:
search(query = "O que é amizade?")

Input question: O que é amizade?
Top-3 lexical search (BM25) hits
	13.619	 Assim que conseguir, faça um esforço para que a amizade se mantenha: promova reuniões, partipe das conversas, mande felicitações nos aniversários e sempre cumpra o que prometeu
	12.666	 O esporte fortaleceu tanto a amizade do grupo que os amigos até já aproveitaram um Carnaval para jogar bola fantasiados de mulher. “Quando você pratica um esporte que ama com os companheiros, tudo é alegria”, conta Rafael
	12.437	 Parece que está falando sobre mim. Fala sobre ver o mundo devagar, com olhos curiosos, conversar com pessoas que não conhece e fazer amizade com a dona da lojinha na esquina de um lugar que você nunca foi...
	12.217	 os pesquisadores pediam que estimassem quão íngreme era a subida e lhes entregavam uma mochila pesada. Os participantes acompanhados apostaram em valores mais baixos do que os que estavam sozinhos – ao lado de um amigo, o obstáculo era menos intimidante. E, quanto mais longa a amizade, meno

In [ ]:
search(query = "O que é amor?")

Input question: O que é amor?
Top-3 lexical search (BM25) hits
	13.645	 É o amor que nos inspira a dar o melhor de nós e a esperar o melhor para os outros
	13.390	 Com ele, criamos vínculos capazes de superar diferenças e distâncias, de vencer o peso do tempo e do cansaço. O amor que recebemos — da família, dos amigos, das paixões, de nós mesmos — nos torna mais fortes, equilibrados, generosos e felizes. O amor que doamos também. Por amor, arriscamos no incerto e acreditamos no futuro
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.247	 Adoro esse instante porque é assim que nos conhecemos melhor e fortalecemos o amor que nos une
	13.130	 Mas se o amor é universal, a forma como o transformamos em relacionamentos é particular: cada um tem a sua, fruto de sua história, do seu tempo, da pessoa ao lado. Enquanto na família ou entre as amizades encaramos o amor de forma tranqüila e espontânea, tudo muda quando se trata do sentiment

In [ ]:
search(query = "como fazer biscoitos?")

Input question: como fazer biscoitos?
Top-3 lexical search (BM25) hits
	12.413	 Se você for fazer vários biscoitos, uma dica do diâmetro do círculo) e cole-o no centro
	9.724	 não existe nada mais fácil que fazer biscoitos, e por isso mesmo as crianças adoram ajudar. “elas podem usar copinhos para modelar a massa”, sugere a doceira Giuliana Azambuja. “ou fazer bolinhas e usar um garfo para decorá-las.” os pequenos também podem participar acrescentando ingredientes à mistura, como pedacinhos de chocolate. “A ajuda na cozinha vira uma grande diversão”, diz Giuliana
	9.466	 Algumas sugestões: biscoitos integrais, iogurte com granola ou um sanduíche
	9.337	 Quebre os biscoitos em pedaços menores e umedeça-os com a bebida escolhida
	9.295	 Você precisa de: > Batatas cruas > 1 faca sem ponta > Cortadores de biscoitos com formatos variados (desses que parecem forminhas) > Tinta de tecido, guache ou almofada para carimbos de várias cores > Papel ou tecido para estampar Como fazer: 1. Corte a b

# Saving

In [ ]:
ENBEDDING_PATH = 'corpus_embedding.pkl'
file = open(ENBEDDING_PATH, 'wb')
pickle.dump(corpus_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)
file.close()
CORPUS_PATH = 'corpus.pkl'
file = open(CORPUS_PATH, 'wb')
pickle.dump(corpus, file, protocol=pickle.HIGHEST_PROTOCOL)
file.close()